In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Try to import UCI ML repo, fallback to manual loading if needed
try:
    from ucimlrepo import fetch_ucirepo
    UCI_AVAILABLE = True
except ImportError:
    print("ucimlrepo not available. Will use manual dataset loading.")
    UCI_AVAILABLE = False

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("=== Advanced Data Mining Project - Deliverable 1 ===")
print("Data Collection, Cleaning, and Exploration")
print("=" * 60)

# ============================================================================
# STEP 1: DATA COLLECTION AND DATASET SELECTION
# ============================================================================

print("\n1. DATASET SELECTION AND JUSTIFICATION")
print("-" * 50)

"""
Dataset Selection: UCI Online Retail Dataset
Source: UCI Machine Learning Repository (ID: 352)
https://archive.ics.uci.edu/dataset/352/online+retail

Dataset Description:
This dataset contains transactional data from a UK-based non-store online retail company
operating between 01/12/2010 and 09/12/2011. The company primarily sells unique 
all-occasion gifts with many wholesale customers.

Dataset Characteristics:
- Records: 541,909 transactions
- Attributes: 8 features including customer, product, and transaction details
- Time span: 11 months of retail operations
- Geographic scope: Multiple countries with UK focus

Justification for Dataset Selection:
1. Size requirements: 541K+ records with 8 attributes exceeds project requirements
2. Real-world business data: Authentic e-commerce transactions
3. Rich analytical opportunities: Customer segmentation, sales forecasting, market basket analysis
4. Multiple modeling possibilities: Supports regression, classification, clustering, and association rules
5. Data quality challenges: Contains missing values and cancellations for cleaning practice
6. Industry relevance: E-commerce analytics is highly valuable in current market
"""

# Load UCI Online Retail Dataset
print("Loading UCI Online Retail Dataset...")

if UCI_AVAILABLE:
    try:
        online_retail = fetch_ucirepo(id=352)
        df = online_retail.data.features.copy()
        print(f"Dataset loaded successfully from UCI repository")
    except Exception as e:
        print(f"UCI repository access failed: {e}")
        print("Please download the dataset manually from:")
        print("https://archive.ics.uci.edu/dataset/352/online+retail")
        print("Save as 'Online_Retail.xlsx' in the current directory")
        exit()
else:
    # Try to load from local file
    try:
        df = pd.read_excel('Online_Retail.xlsx')
        print("Dataset loaded from local Excel file")
    except FileNotFoundError:
        print("Dataset file not found. Please download from:")
        print("https://archive.ics.uci.edu/dataset/352/online+retail")
        print("Save as 'Online_Retail.xlsx' in the current directory")
        exit()
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

print(f"Dataset contains {len(df)} records and {len(df.columns)} attributes")

# ============================================================================
# STEP 2: INITIAL DATA INSPECTION
# ============================================================================

print("\n2. INITIAL DATA INSPECTION")
print("-" * 50)

print("Dataset shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())

print("\nDataset info:")
print(df.info())

print("\nBasic statistics:")
print(df.describe())

print("\nData types:")
print(df.dtypes)

print("\nMissing values:")
print(df.isnull().sum())

print("\nUnique values per column:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

# Check for negative quantities and cancellations
print(f"\nTransactions with negative quantities: {(df['Quantity'] < 0).sum()}")
print(f"Cancellation transactions (InvoiceNo starts with 'C'): {df['InvoiceNo'].astype(str).str.startswith('C').sum()}")

# ============================================================================
# STEP 3: DATA CLEANING
# ============================================================================

print("\n3. DATA CLEANING")
print("-" * 50)

# Store original dataset info for comparison
original_shape = df.shape
print(f"Original dataset shape: {original_shape}")

# 3.1 Handle Missing Values
print("\n3.1 Handling Missing Values:")
print("Missing values before cleaning:")
missing_summary = df.isnull().sum()
print(missing_summary[missing_summary > 0])

# CustomerID has missing values - these represent transactions without customer identification
missing_customers = df['CustomerID'].isnull().sum()
print(f"Transactions without CustomerID: {missing_customers} ({missing_customers/len(df)*100:.1f}%)")

# For customer-focused analysis, remove transactions without CustomerID
df_with_customers = df.dropna(subset=['CustomerID']).copy()
print(f"Dataset after removing missing CustomerID: {df_with_customers.shape}")

# Check for missing descriptions
missing_descriptions = df_with_customers['Description'].isnull().sum()
if missing_descriptions > 0:
    print(f"Transactions with missing Description: {missing_descriptions}")
    df_with_customers = df_with_customers.dropna(subset=['Description'])
    print(f"Dataset after removing missing descriptions: {df_with_customers.shape}")

# 3.2 Handle Cancellations and Returns
print("\n3.2 Handling Cancellations and Returns:")
cancellations = df_with_customers['InvoiceNo'].astype(str).str.startswith('C')
print(f"Cancellation transactions: {cancellations.sum()}")

# Separate cancellations for analysis but keep for now
df_with_customers['is_cancellation'] = cancellations

# Handle negative quantities
negative_qty = df_with_customers['Quantity'] < 0
print(f"Transactions with negative quantities: {negative_qty.sum()}")

# For initial analysis, focus on positive transactions
df_positive = df_with_customers[df_with_customers['Quantity'] > 0].copy()
print(f"Dataset with positive quantities only: {df_positive.shape}")

# 3.3 Handle Data Inconsistencies
print("\n3.3 Handling Data Inconsistencies:")

# Check for zero unit prices
zero_prices = (df_positive['UnitPrice'] <= 0).sum()
print(f"Transactions with zero or negative unit prices: {zero_prices}")

if zero_prices > 0:
    df_positive = df_positive[df_positive['UnitPrice'] > 0]
    print(f"Dataset after removing zero prices: {df_positive.shape}")

# Clean description field
df_positive['Description'] = df_positive['Description'].str.strip().str.upper()

# Convert data types
df_positive['CustomerID'] = df_positive['CustomerID'].astype(int)
df_positive['InvoiceDate'] = pd.to_datetime(df_positive['InvoiceDate'])

# 3.4 Feature Engineering
print("\n3.4 Feature Engineering:")

# Create total amount per transaction line
df_positive['TotalAmount'] = df_positive['Quantity'] * df_positive['UnitPrice']

# Extract date components
df_positive['Year'] = df_positive['InvoiceDate'].dt.year
df_positive['Month'] = df_positive['InvoiceDate'].dt.month
df_positive['DayOfWeek'] = df_positive['InvoiceDate'].dt.dayofweek
df_positive['Hour'] = df_positive['InvoiceDate'].dt.hour

print(f"Added TotalAmount and temporal features")

# 3.5 Handle Outliers
print("\n3.5 Outlier Detection and Treatment:")

def detect_outliers_iqr(data, column):
    """Detect outliers using IQR method"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Check for outliers in key numerical columns
numerical_cols = ['Quantity', 'UnitPrice', 'TotalAmount']
outlier_summary = {}

for col in numerical_cols:
    outliers, lower, upper = detect_outliers_iqr(df_positive, col)
    outlier_summary[col] = {
        'count': len(outliers),
        'percentage': len(outliers) / len(df_positive) * 100,
        'lower_bound': lower,
        'upper_bound': upper
    }
    print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(df_positive)*100:.1f}%)")

# Cap extreme outliers for UnitPrice and TotalAmount
df_cleaned = df_positive.copy()
for col in ['UnitPrice', 'TotalAmount']:
    Q99 = df_cleaned[col].quantile(0.99)
    extreme_outliers = df_cleaned[col] > Q99
    print(f"Capping {extreme_outliers.sum()} extreme outliers in {col} at {Q99:.2f}")
    df_cleaned.loc[df_cleaned[col] > Q99, col] = Q99

print(f"\nFinal cleaned dataset shape: {df_cleaned.shape}")
print(f"Data cleaning summary:")
print(f"- Started with {original_shape[0]} transactions")
print(f"- Removed {original_shape[0] - len(df_cleaned)} problematic records")
print(f"- Final dataset: {len(df_cleaned)} valid transactions")

# Update main dataframe
df = df_cleaned.copy()

# ============================================================================
# STEP 4: EXPLORATORY DATA ANALYSIS (EDA)
# ============================================================================

print("\n4. EXPLORATORY DATA ANALYSIS")
print("-" * 50)

# 4.1 Temporal Analysis
print("\n4.1 Temporal Analysis")

plt.figure(figsize=(15, 10))

# Monthly sales trend
plt.subplot(2, 3, 1)
monthly_sales = df.groupby(['Year', 'Month'])['TotalAmount'].sum().reset_index()
monthly_sales['YearMonth'] = monthly_sales['Year'].astype(str) + '-' + monthly_sales['Month'].astype(str).str.zfill(2)
plt.plot(range(len(monthly_sales)), monthly_sales['TotalAmount'])
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales Amount')
plt.xticks(range(0, len(monthly_sales), 2), monthly_sales['YearMonth'][::2], rotation=45)

# Daily transaction pattern
plt.subplot(2, 3, 2)
daily_pattern = df.groupby('DayOfWeek')['InvoiceNo'].nunique()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
plt.bar(days, daily_pattern.values)
plt.title('Transactions by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Number of Transactions')

# Hourly transaction pattern
plt.subplot(2, 3, 3)
hourly_pattern = df.groupby('Hour')['InvoiceNo'].nunique()
plt.bar(hourly_pattern.index, hourly_pattern.values)
plt.title('Transactions by Hour of Day')
plt.xlabel('Hour')
plt.ylabel('Number of Transactions')

# Quantity distribution
plt.subplot(2, 3, 4)
plt.hist(df['Quantity'], bins=50, alpha=0.7, edgecolor='black')
plt.title('Quantity Distribution')
plt.xlabel('Quantity')
plt.ylabel('Frequency')
plt.xlim(0, df['Quantity'].quantile(0.95))

# Unit price distribution
plt.subplot(2, 3, 5)
plt.hist(df['UnitPrice'], bins=50, alpha=0.7, edgecolor='black')
plt.title('Unit Price Distribution')
plt.xlabel('Unit Price (GBP)')
plt.ylabel('Frequency')
plt.xlim(0, df['UnitPrice'].quantile(0.95))

# Total amount distribution
plt.subplot(2, 3, 6)
plt.hist(df['TotalAmount'], bins=50, alpha=0.7, edgecolor='black')
plt.title('Total Amount Distribution')
plt.xlabel('Total Amount (GBP)')
plt.ylabel('Frequency')
plt.xlim(0, df['TotalAmount'].quantile(0.95))

plt.tight_layout()
plt.show()

# 4.2 Customer Analysis
print("\n4.2 Customer Analysis")

# Customer metrics
customer_metrics = df.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',  # Number of orders
    'Quantity': 'sum',       # Total items purchased
    'TotalAmount': 'sum',    # Total spent
    'InvoiceDate': ['min', 'max']  # First and last purchase
}).round(2)

customer_metrics.columns = ['OrderCount', 'TotalQuantity', 'TotalSpent', 'FirstPurchase', 'LastPurchase']
customer_metrics['CustomerLifespan'] = (customer_metrics['LastPurchase'] - customer_metrics['FirstPurchase']).dt.days
customer_metrics['AvgOrderValue'] = customer_metrics['TotalSpent'] / customer_metrics['OrderCount']

plt.figure(figsize=(15, 10))

# Customer order count distribution
plt.subplot(2, 3, 1)
plt.hist(customer_metrics['OrderCount'], bins=30, alpha=0.7, edgecolor='black')
plt.title('Customer Order Count Distribution')
plt.xlabel('Number of Orders')
plt.ylabel('Number of Customers')

# Customer total spent distribution
plt.subplot(2, 3, 2)
plt.hist(customer_metrics['TotalSpent'], bins=30, alpha=0.7, edgecolor='black')
plt.title('Customer Total Spending Distribution')
plt.xlabel('Total Spent (GBP)')
plt.ylabel('Number of Customers')
plt.xlim(0, customer_metrics['TotalSpent'].quantile(0.95))

# Average order value distribution
plt.subplot(2, 3, 3)
plt.hist(customer_metrics['AvgOrderValue'], bins=30, alpha=0.7, edgecolor='black')
plt.title('Average Order Value Distribution')
plt.xlabel('Average Order Value (GBP)')
plt.ylabel('Number of Customers')
plt.xlim(0, customer_metrics['AvgOrderValue'].quantile(0.95))

# Customer lifespan distribution
plt.subplot(2, 3, 4)
plt.hist(customer_metrics['CustomerLifespan'], bins=30, alpha=0.7, edgecolor='black')
plt.title('Customer Lifespan Distribution')
plt.xlabel('Days Between First and Last Purchase')
plt.ylabel('Number of Customers')

# Top countries by sales
plt.subplot(2, 3, 5)
country_sales = df.groupby('Country')['TotalAmount'].sum().sort_values(ascending=False).head(10)
plt.bar(range(len(country_sales)), country_sales.values)
plt.title('Top 10 Countries by Sales')
plt.xlabel('Country')
plt.ylabel('Total Sales (GBP)')
plt.xticks(range(len(country_sales)), country_sales.index, rotation=45)

# RFM Analysis preparation
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalAmount': 'sum'
}).round(2)
rfm.columns = ['Recency', 'Frequency', 'Monetary']

plt.subplot(2, 3, 6)
plt.scatter(rfm['Recency'], rfm['Monetary'], alpha=0.6)
plt.title('RFM Analysis: Recency vs Monetary')
plt.xlabel('Recency (Days)')
plt.ylabel('Monetary Value (GBP)')

plt.tight_layout()
plt.show()

# 4.3 Product Analysis
print("\n4.3 Product Analysis")

# Product metrics
product_metrics = df.groupby('StockCode').agg({
    'Description': 'first',
    'Quantity': 'sum',
    'TotalAmount': 'sum',
    'CustomerID': 'nunique'
}).round(2)
product_metrics.columns = ['Description', 'TotalQuantitySold', 'TotalRevenue', 'UniqueCustomers']

plt.figure(figsize=(15, 10))

# Top products by quantity
plt.subplot(2, 2, 1)
top_qty_products = product_metrics.nlargest(10, 'TotalQuantitySold')
plt.barh(range(len(top_qty_products)), top_qty_products['TotalQuantitySold'])
plt.title('Top 10 Products by Quantity Sold')
plt.xlabel('Total Quantity Sold')
plt.yticks(range(len(top_qty_products)), top_qty_products['Description'].str[:30])

# Top products by revenue
plt.subplot(2, 2, 2)
top_revenue_products = product_metrics.nlargest(10, 'TotalRevenue')
plt.barh(range(len(top_revenue_products)), top_revenue_products['TotalRevenue'])
plt.title('Top 10 Products by Revenue')
plt.xlabel('Total Revenue (GBP)')
plt.yticks(range(len(top_revenue_products)), top_revenue_products['Description'].str[:30])

# Product popularity (unique customers)
plt.subplot(2, 2, 3)
top_popular_products = product_metrics.nlargest(10, 'UniqueCustomers')
plt.barh(range(len(top_popular_products)), top_popular_products['UniqueCustomers'])
plt.title('Top 10 Products by Customer Reach')
plt.xlabel('Number of Unique Customers')
plt.yticks(range(len(top_popular_products)), top_popular_products['Description'].str[:30])

# Price vs Quantity relationship
plt.subplot(2, 2, 4)
plt.scatter(df['UnitPrice'], df['Quantity'], alpha=0.6)
plt.title('Unit Price vs Quantity Relationship')
plt.xlabel('Unit Price (GBP)')
plt.ylabel('Quantity')
plt.xlim(0, df['UnitPrice'].quantile(0.95))
plt.ylim(0, df['Quantity'].quantile(0.95))

plt.tight_layout()
plt.show()

# 4.4 Correlation Analysis
print("\n4.4 Correlation Analysis")

# Select numerical columns for correlation
numerical_df = df[['Quantity', 'UnitPrice', 'TotalAmount', 'Year', 'Month', 'DayOfWeek', 'Hour']].copy()
correlation_matrix = numerical_df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix of Numerical Variables')
plt.tight_layout()
plt.show()

print("Key correlations found:")
strong_corr = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.3:
            strong_corr.append((correlation_matrix.columns[i], 
                              correlation_matrix.columns[j], corr_val))

for var1, var2, corr in strong_corr:
    print(f"- {var1} vs {var2}: {corr:.3f}")

# ============================================================================
# STEP 5: KEY INSIGHTS AND SUMMARY
# ============================================================================

print("\n5. KEY INSIGHTS AND SUMMARY")
print("-" * 50)

print("\n5.1 Dataset Summary:")
print(f"- Final dataset contains {len(df)} transaction records")
print(f"- Covers {df['CustomerID'].nunique()} unique customers")
print(f"- Includes {df['StockCode'].nunique()} unique products")
print(f"- Spans {df['Country'].nunique()} countries")
print(f"- Time period: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")

print("\n5.2 Business Performance Insights:")
total_revenue = df['TotalAmount'].sum()
avg_order_value = df.groupby('InvoiceNo')['TotalAmount'].sum().mean()
total_transactions = df['InvoiceNo'].nunique()

print(f"- Total revenue: £{total_revenue:,.2f}")
print(f"- Total transactions: {total_transactions:,}")
print(f"- Average order value: £{avg_order_value:.2f}")
print(f"- Average items per transaction: {df.groupby('InvoiceNo')['Quantity'].sum().mean():.1f}")

print("\n5.3 Customer Behavior Insights:")
print(f"- Average customer lifetime: {customer_metrics['CustomerLifespan'].mean():.0f} days")
print(f"- Average orders per customer: {customer_metrics['OrderCount'].mean():.1f}")
print(f"- Average customer value: £{customer_metrics['TotalSpent'].mean():.2f}")
print(f"- Most active country: {df.groupby('Country')['TotalAmount'].sum().idxmax()}")

print("\n5.4 Product Performance Insights:")
best_selling_product = product_metrics.loc[product_metrics['TotalQuantitySold'].idxmax(), 'Description']
highest_revenue_product = product_metrics.loc[product_metrics['TotalRevenue'].idxmax(), 'Description']

print(f"- Best-selling product by quantity: {best_selling_product}")
print(f"- Highest revenue product: {highest_revenue_product}")
print(f"- Average product price: £{df['UnitPrice'].mean():.2f}")

print("\n5.5 Temporal Patterns:")
peak_month = monthly_sales.loc[monthly_sales['TotalAmount'].idxmax(), 'YearMonth']
peak_day = days[daily_pattern.idxmax()]
peak_hour = hourly_pattern.idxmax()

print(f"- Peak sales month: {peak_month}")
print(f"- Most active day: {peak_day}")
print(f"- Peak transaction hour: {peak_hour}:00")
print(f"- Sales seasonality: Strong November/December peak visible")

print("\n5.6 Data Quality Assessment:")
print(f"- Data completeness: {(1 - df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100:.1f}%")
print(f"- Customer identification rate: {len(df) / original_shape[0] * 100:.1f}%")
print(f"- Transaction value validity: 100% (after cleaning)")

print("\n5.7 Implications for Future Modeling:")
print("- Customer Segmentation: Clear RFM patterns for customer classification")
print("- Sales Forecasting: Strong temporal patterns for time series modeling")
print("- Market Basket Analysis: Rich product transaction data available")
print("- Churn Prediction: Customer purchase frequency and recency patterns")
print("- Revenue Optimization: Price-quantity relationships identified")
print("- Geographic Analysis: Multi-country sales patterns for expansion modeling")

print("\n5.8 Recommended Feature Engineering for Next Deliverables:")
print("- Customer lifetime value calculations")
print("- RFM score creation for segmentation")
print("- Product category clustering")
print("- Seasonal adjustment factors")
print("- Customer churn indicators")
print("- Market basket association metrics")

# ============================================================================
# FINAL DATASET EXPORT
# ============================================================================

print("\n6. DATASET EXPORT")
print("-" * 50)

# Save cleaned dataset
output_filename = 'cleaned_online_retail_dataset.csv'
df.to_csv(output_filename, index=False)
print(f"Cleaned dataset exported to: {output_filename}")

# Save customer metrics for future use
customer_metrics.to_csv('customer_metrics.csv')
print(f"Customer metrics exported to: customer_metrics.csv")

# Save RFM analysis
rfm.to_csv('rfm_analysis.csv')
print(f"RFM analysis exported to: rfm_analysis.csv")